In [1]:
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn import metrics
import tensorflow as tf
import pandas as pd
import numpy as np
import plotting
import hls4ml
import shap
import sys
import os

np.random.seed(77)

import matplotlib.pyplot as plt
import mplhep
plt.style.use(mplhep.style.CMS)


def print_dict(d, indent=0):
    align=20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))

2023-05-31 19:33:30.623668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-05-31 19:33:30.623701: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/llr/cms/motta/.local/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes,

In [2]:
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/settings64.sh
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/settings64.sh

In [3]:
options = {
    'v'          : '16',
    'date'       : '2023_05_28',
    'inTagIdent' : '_QP',
    'inTagCalib' : '_QP',
    'caloClNxM'  : '5x9',
    'sparsity'   : 0.5
}

In [4]:
indir = '/data_CMS/cms/motta/Phase2L1T/'+options['date']+'_v'+options['v']+'/TauMinator_CE_cltw'+options['caloClNxM']+'_Training'+options['inTagIdent']

N = int(options['caloClNxM'].split('x')[0])
M = int(options['caloClNxM'].split('x')[1])

sparsityTag = str(options['sparsity']).split('.')[0]+'p'+str(options['sparsity']).split('.')[1]

**Only models with one input tensor are currently supported by VivadoAcceleratorBackend.
Therefore, the CNN model is in this case a special version that takes only the image as an input and gives as an output the flattened tensor out of the convolutions withou concatenating the position of the cluster.**

In [5]:
# quantized and pruned models
QCNN = keras.models.load_model(indir+'/CNNmodel_special4accelerator', compile=False)
QDNNident = keras.models.load_model(indir+'/ID_DNNmodel', compile=False)
QDNNcalib = keras.models.load_model(indir+'/CAL_DNNmodel', compile=False)

2023-05-31 19:33:37.292501: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /grid_mnt/opt__exp_soft/llr/python/3.9.9/lib/python3.9/site-packages/cv2/../../lib64:/opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-05-31 19:33:37.292550: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-31 19:33:37.292585: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (llrgrhgtrig.in2p3.fr): /proc/driver/nvidia/version does not exist
2023-05-31 19:33:37.292985: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical

# QUANTIZED MODELS ONLY

In [6]:
############################## Pass quantized CNN model through hls4ml ##############################

synth = True
if synth:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    QCNN_hls_cfg = hls4ml.utils.config_from_keras_model(QCNN, granularity='name')
    QCNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    QCNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in QCNN_hls_cfg['LayerName'].keys():
        QCNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
        QCNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        QCNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    QCNN_hls_cfg['LayerName']['TowerClusterImage']['Precision'] = 'ap_ufixed<10,8>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['ParallelizationFactor'] = 4
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<14,7>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<14,7>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<10,7>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<10,7>'

    QCNN_hls_cfg['LayerName']['MP_CNNpBNlayer1']['Precision'] = 'ap_fixed<10,7>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<14,6>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<14,6>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<9,6>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<9,6>'

    print(QCNN_hls_cfg)

    QCNN_hls = hls4ml.converters.convert_from_keras_model(QCNN,
                                                          hls_config=QCNN_hls_cfg,
                                                          output_dir=indir+'/CNNmodel_HLS_XCU200deployment/',
                                                          backend='VivadoAccelerator',
                                                          board='alveo-u200',
                                                          io_type='io_parallel',
                                                          platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QCNN_hls.compile()

    QCNN_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=False)

Interpreting Model
Topology:
Layer name: TowerClusterImage, layer type: Input
Layer name: CNNpBNlayer1, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer1
Layer name: RELU_CNNpBNlayer1, layer type: QActivation
  -> Activation (quantized_relu(10,7)), layer name: RELU_CNNpBNlayer1
Layer name: MP_CNNpBNlayer1, layer type: MaxPooling2D
Layer name: CNNpBNlayer2, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer2
Layer name: RELU_CNNpBNlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CNNpBNlayer2
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'TowerClusterImage': {'Precision': 'ap_ufixed<10,8>', 'Strategy': 'Latency', 'ReuseFactor': 1, 'Trace': True}, 'CNNpBNlayer1': {'Precision': {'weight': 'ap_fixed<5,1>', 'bias': 'ap_fixed<6,1>', 'accum': 'ap_fixed<14,7>', 'result': 'ap_fixed<14,7>'}, 'ReuseFactor': 1, 'Strategy': 'Latency', 'Trace': True, 'Pa

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_activation.h:69) in function 'nnet::relu<ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, relu_config3>' completely with a factor of 128.
INFO: [HLS 200-489] Unrolling loop 'PixelLoop' (firmware/nnet_utils/nnet_conv2d_latency.h:202) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 4.
INFO: [HLS 200-489] Unrolling loop 'Product1' (firmware/nnet_utils/nnet_conv2d_latency.h:208) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 12.
INFO: [HLS 200-489] Unrolling loop 'Product2' (firmware/nnet_utils/nnet_conv2d_latency.h:210) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>

INFO: [XFORM 203-401] Performing if-conversion on hyperblock from (firmware/nnet_utils/nnet_activation.h:64:18) to (firmware/nnet_utils/nnet_activation.h:74:1) in function 'nnet::relu<ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, relu_config3>'... converting 257 basic blocks.
INFO: [XFORM 203-401] Performing if-conversion on hyperblock from (firmware/nnet_utils/nnet_activation.h:64:18) to (firmware/nnet_utils/nnet_activation.h:74:1) in function 'nnet::relu<ap_fixed<14, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, relu_config6>'... converting 49 basic blocks.
INFO: [XFORM 203-401] Performing if-conversion on hyperblock to (firmware/nnet_utils/nnet_code_gen.h:98:5) in function 'nnet::fill_buffer_2<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>::fill_buffer'... converting 17 basic blocks.
INFO: [XFORM 203-401] Performing if-conversion on hyperblock from (firmware/myproject_axi.cpp:16:40) to (

INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.23 seconds; current allocated memory: 616.639 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'conv_2d_cl_ap_fixed_ap_fixed_14_6_5_3_0_config5_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining loop 'PartitionLoop'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 1.13 seconds; current allocated memory: 618.543 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIN

...

INFO: [Synth 8-256] done synthesizing module 'conv_2d_cl_ap_ufixed_ap_fixed_14_7_5_3_0_config2_s' (4#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/conv_2d_cl_ap_ufixed_ap_fixed_14_7_5_3_0_config2_s.vhd:153]
INFO: [Synth 8-638] synthesizing module 'relu_ap_fixed_ap_fixed_10_7_0_0_0_relu_config3_s' [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/relu_ap_fixed_ap_fixed_10_7_0_0_0_relu_config3_s.vhd:280]
INFO: [Synth 8-256] done synthesizing module 'relu_ap_fixed_ap_fixed_10_7_0_0_0_relu_config3_s' (5#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/relu_ap_fixed_ap_fixed_10_7_0_0_0_relu_config3_s.vhd:280]
INFO: [Synth 8-638] synthesizing module 'poo

INFO: [Synth 8-256] done synthesizing module 'conv_2d_cl_ap_fixed_ap_fixed_14_6_5_3_0_config5_s' (11#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/conv_2d_cl_ap_fixed_ap_fixed_14_6_5_3_0_config5_s.vhd:80]
INFO: [Synth 8-638] synthesizing module 'relu_ap_fixed_ap_fixed_9_6_0_0_0_relu_config6_s' [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/relu_ap_fixed_ap_fixed_9_6_0_0_0_relu_config6_s.vhd:96]
INFO: [Synth 8-256] done synthesizing module 'relu_ap_fixed_ap_fixed_9_6_0_0_0_relu_config6_s' (12#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/relu_ap_fixed_ap_fixed_9_6_0_0_0_relu_config6_s.vhd:96]
INFO: [Synth 8-638] synthesizing module 'fifo_w14_d

Part Resources:
DSPs: 6840 (col length:120)
BRAMs: 4320 (col length: RAMB18 360 RAMB36 180)
---------------------------------------------------------------------------------
Finished Part Resource Summary
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start Cross Boundary and Area Optimization
---------------------------------------------------------------------------------
INFO: [Synth 8-3886] merging instance 'p_1_i_idx1_1_313_t_i_reg_81773_reg[4]' (FDE) to 'p_1_i_idx1_3_115_t_i_reg_82047_reg[4]'
INFO: [Synth 8-3886] merging instance 'p_1_i_idx1_1_313_t_i_reg_81773_reg[5]' (FDE) to 'p_1_i_idx1_3_115_t_i_reg_82047_reg[5]'
INFO: [Synth 8-3886] merging instance 'p_1_i_idx1_1_313_t_i_reg_81773_reg[6]' (FDE) to 'p_1_i_idx1_3_115_t_i_reg_82047_reg[6]'
INFO: [Synth 8-4471] merging register 'data_buf_i_3_2_i_reg_81291_reg[9:0]' into 'data_buf_i_3_2_i_reg_81291_reg[9:0]' [/grid

No constraint files found.
---------------------------------------------------------------------------------
Start Timing Optimization
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Timing Optimization : Time (s): cpu = 00:01:24 ; elapsed = 00:02:59 . Memory (MB): peak = 3382.555 ; gain = 1123.047 ; free physical = 40947 ; free virtual = 323133
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start Technology Mapping
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Technology Mapping : Time (s): cpu = 00:01:28 ; elapsed = 00:03:04 . Memory (MB): peak = 3390.559 ; gain = 1131.051 ; free physical = 40951 ; free virtual = 323142
-

Synthesis Optimization Runtime : Time (s): cpu = 00:01:41 ; elapsed = 00:03:20 . Memory (MB): peak = 3390.559 ; gain = 1131.051 ; free physical = 47191 ; free virtual = 329407
Synthesis Optimization Complete : Time (s): cpu = 00:01:41 ; elapsed = 00:03:20 . Memory (MB): peak = 3390.559 ; gain = 1131.051 ; free physical = 47198 ; free virtual = 329407
INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.25 ; elapsed = 00:00:00.25 . Memory (MB): peak = 3390.770 ; gain = 0.000 ; free physical = 47166 ; free virtual = 329375
INFO: [Netlist 29-17] Analyzing 1957 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00 ; elapsed = 00:00:00.01 . Memory (MB): peak = 3441.758 ; gain = 0.000 ; free physical = 47087 ; free virtual 

In [7]:
############################## Pass quantized identification DNN model through hls4ml ##############################

synth = True
if synth:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    id_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNident, granularity='name')
    id_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    id_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in id_QDNN_hls_cfg['LayerName'].keys():
        id_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        id_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        id_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    id_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['accum'] = 'ap_fixed<13,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['result'] = 'ap_fixed<8,6>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer1']['Precision'] = 'ap_fixed<9,5>'

    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['accum'] = 'ap_fixed<11,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['result'] = 'ap_fixed<10,5>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer2']['Precision'] = 'ap_fixed<9,5>'

    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['weight'] = 'ap_fixed<2,1>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['accum'] = 'ap_fixed<7,4>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['result'] = 'ap_fixed<7,5>'

    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Precision'] = 'ap_fixed<8,1>'
    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Strategy'] = 'Stable'

    print(id_QDNN_hls_cfg)

    QDNNident_hls = hls4ml.converters.convert_from_keras_model(QDNNident,
                                                               hls_config=id_QDNN_hls_cfg,
                                                               output_dir=indir+'/ID_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_stream',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNident_hls.compile()
    
    QDNNident_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=False)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: IDlayer1, layer type: QDense
  -> Activation (linear), layer name: IDlayer1
Layer name: RELU_IDlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_IDlayer1
Layer name: IDlayer2, layer type: QDense
  -> Activation (linear), layer name: IDlayer2
Layer name: RELU_IDlayer2, layer type: QActivation
  -> Activation (quantized_relu(8,5)), layer name: RELU_IDlayer2
Layer name: IDout, layer type: QDense
  -> Activation (linear), layer name: IDout
Layer name: sigmoid_IDout, layer type: Activation
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'IDlayer1': {'Precision': {'weight': 'ap_fixed<3,1>', 'accum': 'ap_fixed<13,6>', 'result': 'ap_fixed<8,6>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'IDlayer1_linear': {

INFO: [XFORM 203-603] Inlining function 'nnet::array<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 34u>::operator[]' into 'myproject_axi' (firmware/myproject_axi.cpp:24).
INFO: [XFORM 203-603] Inlining function 'nnet::array<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 34u>::operator[]' into 'nnet::dense<nnet::array<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 34u>, nnet::array<ap_fixed<8, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config2>' (firmware/nnet_utils/nnet_dense_stream.h:48).
INFO: [XFORM 203-603] Inlining function 'nnet::product::mult<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<6, 1, (ap_q_mode)5, (ap_o_mode)3, 0> >::limit' into 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<8, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_dense_latency.h:56).
INFO: [XFORM 203-603] Inlining function 'nnet::product::mult<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<6, 1, (ap_q_mode)5, (ap_o_mode)3, 0> >::pr

INFO: [HLS 200-111] Finished Standard Transforms Time (s): cpu = 00:00:33 ; elapsed = 00:00:37 . Memory (MB): peak = 1760.188 ; gain = 1231.879 ; free physical = 49884 ; free virtual = 331860
INFO: [HLS 200-10] Checking synthesizability ...
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.

INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_dense_latency.h:68) in function 'nnet::dense_wrapper<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<10, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_dense_latency.h:73) in function 'nnet::dense_wrapper<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<10, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_dense_latency.h:74) in function 'nnet::dense_wrapper<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<10, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_latency.h:81) in function 'nnet::dense_wrapper<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<10, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, 

INFO: [XFORM 203-712] Applying dataflow to function 'myproject', detected/extracted 6 process function(s): 
	 'nnet::dense<nnet::array<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 34u>, nnet::array<ap_fixed<8, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config2>'
	 'nnet::relu<nnet::array<ap_fixed<8, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, nnet::array<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, 16u>, relu_config3>'
	 'nnet::dense<nnet::array<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, 16u>, nnet::array<ap_fixed<10, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, 8u>, config4>'
	 'nnet::relu<nnet::array<ap_fixed<10, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, 8u>, nnet::array<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, 8u>, relu_config5>'
	 'nnet::dense<nnet::array<ap_fixed<9, 5, (ap_q_mode)0, (ap_o_mode)0, 0>, 8u>, nnet::array<ap_fixed<7, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, config6>'
	 'nnet::sigmoid<nnet::array<ap_fixed<7, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, nnet::array<ap_fixed<8, 1, (ap

...

INFO: [Synth 8-256] done synthesizing module 'dense_wrapper_ap_fixed_ap_fixed_8_6_5_3_0_config2_s' (1#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/dense_wrapper_ap_fixed_ap_fixed_8_6_5_3_0_config2_s.vhd:69]
INFO: [Synth 8-256] done synthesizing module 'dense_array_array_ap_fixed_8_6_5_3_0_16u_config2_s' (2#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/dense_array_array_ap_fixed_8_6_5_3_0_16u_config2_s.vhd:177]
INFO: [Synth 8-3491] module 'relu_array_array_ap_fixed_16u_relu_config3_s' declared at '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/relu_array_array_ap_fixed_16u_relu_config3_s.vhd:12' bound to instance 'relu_array_array_ap_

INFO: [Synth 8-3491] module 'fifo_w9_d1_A' declared at '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/fifo_w9_d1_A.vhd:47' bound to instance 'layer5_out_V_data_4_V_U' of component 'fifo_w9_d1_A' [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/myproject.vhd:2199]
INFO: [Synth 8-3491] module 'fifo_w9_d1_A' declared at '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/fifo_w9_d1_A.vhd:47' bound to instance 'layer5_out_V_data_5_V_U' of component 'fifo_w9_d1_A' [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/myproject.vhd:2212]
INFO: [S

INFO: [Synth 8-256] done synthesizing module 'myproject' (30#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/ID_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/myproject.vhd:129]
---------------------------------------------------------------------------------
Finished Synthesize : Time (s): cpu = 00:00:06 ; elapsed = 00:00:11 . Memory (MB): peak = 2259.910 ; gain = 0.000 ; free physical = 48516 ; free virtual = 330564
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Constraint Validation : Time (s): cpu = 00:00:07 ; elapsed = 00:00:11 . Memory (MB): peak = 2259.910 ; gain = 0.000 ; free physical = 48763 ; free virtual = 330810
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start Lo

No constraint files found.
---------------------------------------------------------------------------------
Start Timing Optimization
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Timing Optimization : Time (s): cpu = 00:00:33 ; elapsed = 00:01:05 . Memory (MB): peak = 2992.332 ; gain = 732.422 ; free physical = 47762 ; free virtual = 329820
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start Technology Mapping
---------------------------------------------------------------------------------
INFO: [Synth 8-7052] The timing for the instance sigmoid_array_array_ap_fixed_1u_sigmoid_config7_U0/sigmoid_table4_U/sigmoid_array_array_ap_fixed_1u_sigmoid_config7_s_sigmoidbkb_rom_U/q0_reg (implemented as a Block RAM) might be sub-optimal as no optional o

INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.08 ; elapsed = 00:00:00.09 . Memory (MB): peak = 2996.461 ; gain = 0.000 ; free physical = 47850 ; free virtual = 329908
INFO: [Netlist 29-17] Analyzing 1309 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00 ; elapsed = 00:00:00 . Memory (MB): peak = 3092.141 ; gain = 0.000 ; free physical = 47687 ; free virtual = 329745
INFO: [Project 1-111] Unisim Transformation Summary:
  A total of 425 instances were transformed.
  BUFG => BUFGCE: 1 instance 
  DSP48E2 => DSP48E2 (DSP_ALU, DSP_A_B_DATA, DSP_C_DATA, DSP_MULTIPLIER, DSP_M_DATA, DSP_OUTPUT, DSP_PREADD, DSP_PREADD_DATA): 11 instances
  IBUF => IBUF (IBUFCTRL, INBUF): 413 instances

INFO: [Common 17-83] Releasing

In [8]:
############################## Pass quantized calibration DNN model through hls4ml ##############################

synth = True
if synth:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    cal_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNcalib, granularity='name')
    cal_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    cal_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in cal_QDNN_hls_cfg['LayerName'].keys():
        cal_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        cal_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        cal_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    cal_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['weight'] = 'ap_fixed<6,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['accum'] = 'ap_fixed<16,5>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['result'] = 'ap_fixed<14,6>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer1']['Precision'] = 'ap_fixed<9,6>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['accum'] = 'ap_fixed<16,7>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['result'] = 'ap_fixed<12,8>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer2']['Precision'] = 'ap_fixed<9,7>'

    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['weight'] = 'ap_fixed<2,1>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['accum'] = 'ap_fixed<12,9>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['result'] = 'ap_fixed<10,9>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Strategy'] = 'Stable'

    print(cal_QDNN_hls_cfg)

    QDNNcalib_hls = hls4ml.converters.convert_from_keras_model(QDNNcalib,
                                                               hls_config=cal_QDNN_hls_cfg,
                                                               output_dir=indir+'/CAL_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_stream',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNcalib_hls.compile()
    
    QDNNcalib_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=False)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: CALlayer1, layer type: QDense
  -> Activation (linear), layer name: CALlayer1
Layer name: RELU_CALlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer1
Layer name: CALlayer2, layer type: QDense
  -> Activation (linear), layer name: CALlayer2
Layer name: RELU_CALlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer2
Layer name: CALout, layer type: QDense
  -> Activation (linear), layer name: CALout
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'CALlayer1': {'Precision': {'weight': 'ap_fixed<6,1>', 'accum': 'ap_fixed<16,5>', 'result': 'ap_fixed<14,6>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'CALlayer1_linear': {'Precision': 'ap_fixed<16,6>', 'Reuse

INFO: [HLS 200-111] Finished Standard Transforms Time (s): cpu = 00:00:32 ; elapsed = 00:00:38 . Memory (MB): peak = 1760.188 ; gain = 1231.879 ; free physical = 49711 ; free virtual = 331799
INFO: [HLS 200-10] Checking synthesizability ...
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.

INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_dense_latency.h:68) in function 'nnet::dense_wrapper<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_dense_latency.h:73) in function 'nnet::dense_wrapper<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_dense_latency.h:74) in function 'nnet::dense_wrapper<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_latency.h:81) in function 'nnet::dense_wrapper<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, 

INFO: [XFORM 203-712] Applying dataflow to function 'myproject', detected/extracted 5 process function(s): 
	 'nnet::dense<nnet::array<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 34u>, nnet::array<ap_fixed<14, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config2>'
	 'nnet::relu<nnet::array<ap_fixed<14, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, nnet::array<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, 16u>, relu_config3>'
	 'nnet::dense<nnet::array<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, 16u>, nnet::array<ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, 8u>, config4>'
	 'nnet::relu<nnet::array<ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, 8u>, nnet::array<ap_fixed<9, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, 8u>, relu_config5>'
	 'nnet::dense<nnet::array<ap_fixed<9, 7, (ap_q_mode)0, (ap_o_mode)0, 0>, 8u>, nnet::array<ap_fixed<10, 9, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, config6>'.
INFO: [XFORM 203-712] Applying dataflow to function 'myproject_axi', detected/extracted 3 process function(

INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Starting global binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 1.02 seconds; current allocated memory: 494.979 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'dense_array_array_ap_fixed_14_6_5_3_0_16u_config2_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 1.2 seconds; current allocated memory: 495.660 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring reso

...

INFO: [Synth 8-256] done synthesizing module 'start_for_relu_array_array_ap_fixed_16u_relu_config3_U0' (16#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CAL_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/start_for_relu_array_array_ap_fixed_16u_relu_config3_U0.vhd:66]
INFO: [Synth 8-3491] module 'start_for_dense_array_array_ap_fixed_12_8_5_3_0_8u_configbkb' declared at '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CAL_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/start_for_dense_array_array_ap_fixed_12_8_5_3_0_8u_configbkb.vhd:47' bound to instance 'start_for_dense_array_array_ap_fixed_12_8_5_3_0_8u_configbkb_U' of component 'start_for_dense_array_array_ap_fixed_12_8_5_3_0_8u_configbkb' [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CAL_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/

INFO: [Synth 8-256] done synthesizing module 'myproject' (23#1) [/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_05_28_v16/TauMinator_CE_cltw5x9_Training_QP/CAL_DNNmodel_HLS_XCU200deployment/myproject_prj/solution1/syn/vhdl/myproject.vhd:129]
---------------------------------------------------------------------------------
Finished Synthesize : Time (s): cpu = 00:00:06 ; elapsed = 00:00:10 . Memory (MB): peak = 2256.941 ; gain = 0.000 ; free physical = 47629 ; free virtual = 329785
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Constraint Validation : Time (s): cpu = 00:00:06 ; elapsed = 00:00:11 . Memory (MB): peak = 2256.941 ; gain = 0.000 ; free physical = 47629 ; free virtual = 329785
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start L

---------------------------------------------------------------------------------
Finished Cross Boundary and Area Optimization : Time (s): cpu = 00:00:32 ; elapsed = 00:01:03 . Memory (MB): peak = 2991.344 ; gain = 734.402 ; free physical = 46735 ; free virtual = 328900
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start ROM, RAM, DSP and Shift Register Reporting
---------------------------------------------------------------------------------

DSP: Preliminary Mapping	Report (see note below)
+-----------------------------------------------------+---------------+--------+--------+--------+--------+--------+------+------+------+------+-------+------+------+
|Module Name                                          | DSP Mapping   | A Size | B Size | C Size | D Size | P Size | AREG | BREG | CREG | DREG | ADREG | MREG | PREG | 
+-----------------------------------------------

---------------------------------------------------------------------------------
Finished Timing Optimization : Time (s): cpu = 00:00:33 ; elapsed = 00:01:05 . Memory (MB): peak = 2991.348 ; gain = 734.406 ; free physical = 46698 ; free virtual = 328864
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start Technology Mapping
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Finished Technology Mapping : Time (s): cpu = 00:00:35 ; elapsed = 00:01:07 . Memory (MB): peak = 2991.348 ; gain = 734.406 ; free physical = 46658 ; free virtual = 328823
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Start IO Insertion
-------------------------------------

INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.09 ; elapsed = 00:00:00.09 . Memory (MB): peak = 2995.477 ; gain = 0.000 ; free physical = 46780 ; free virtual = 328945
INFO: [Netlist 29-17] Analyzing 1208 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00 ; elapsed = 00:00:00 . Memory (MB): peak = 3103.094 ; gain = 0.000 ; free physical = 46678 ; free virtual = 328844
INFO: [Project 1-111] Unisim Transformation Summary:
  A total of 452 instances were transformed.
  BUFG => BUFGCE: 1 instance 
  DSP48E2 => DSP48E2 (DSP_ALU, DSP_A_B_DATA, DSP_C_DATA, DSP_MULTIPLIER, DSP_M_DATA, DSP_OUTPUT, DSP_PREADD, DSP_PREADD_DATA): 38 instances
  IBUF => IBUF (IBUFCTRL, INBUF): 413 instances

INFO: [Common 17-83] Releasing